In [1]:
#Enable autoreloading of imported modules
%load_ext autoreload
%autoreload 2

#Import required packages
import sys,os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.sparse as sp

#Add the repo root (one level up from this notebook) to sys.path
sys.path.insert(0, os.path.abspath("../"))

In [2]:
#Install necessary resources from nltk
import nltk

nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\phili\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\phili\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\phili\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## 1. Download data

In [3]:
import kagglehub

#Download latest version of dataset
print("Load or download dataset...")
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset") #path to downloaded dataset
#   (if already downloaded, will not download again)

fake_path=os.path.join(path, "Fake.csv")    #path to dataset with true news
true_path=os.path.join(path, "True.csv")    #path to dataset with fake news

#Read into dataframes
print("Loading fake.csv ...")
fake_df=pd.read_csv(fake_path)
print("Loading true.csv ...")
true_df=pd.read_csv(true_path)

#Label data (1=true, 0=fakenews)
true_df['label']=1
fake_df['label']=0

#Join dataframes
df=pd.concat([true_df, fake_df])
print("Done")


Load or download dataset...
Loading fake.csv ...
Loading true.csv ...
Done


## 2. Preprocess data

In [4]:
from courselib.utils.splits import train_test_split

training_data_fraction=.8

#Split
df_, train_df, test_df=train_test_split(df, training_data_fraction=training_data_fraction, class_column_name='label', return_numpy=False)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from courselib.utils.normalization import standardize_sparse_matrix

column='title' #column of df to vectorize


#Vectorize train, test split
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform((train_df[column]))
X_test=vectorizer.transform((test_df[column]))
X=sp.vstack([X_train, X_test])

Y_train=train_df['label'].to_numpy()
Y_test=test_df['label'].to_numpy()

#-1, 1 labels for SVM
Y_train_neg=Y_train.copy()
Y_train_neg[Y_train_neg==0]=-1
Y_test_neg=Y_test.copy()
Y_test_neg[Y_test_neg==0]=-1

#Apply additional z-score normalization (works better for svm)
X, offset=standardize_sparse_matrix(X)
X_train, X_test=X[:len(train_df)], X[len(train_df):]

In [6]:
num_features=X_train.shape[1] #number of extracted features from train_df[column]
num_features

19619

## 3. Comparison of different models

In [7]:
# from courselib.models.glm import LogisticRegression
from courselib.models.svm import LinearSVM, BinaryKernelSVM


#Initialize models
w=np.zeros(num_features)

ImportError: cannot import name 'TrainableModel' from 'courselib.models.base' (c:\Users\phili\Documents\Programmieren-Uni\AppliedML\FakeNewsProject\FakeNewsDetection_Project\courselib\models\base.py)

In [ ]:




from extensions.vectorization_and_tokenization import multi_column_vectorizer, basic_word_tokenizer, lemmatization_tokenizer, stemming_tokenizer
from courselib.utils.metrics import binary_accuracy, accuracy
from extensions.sparse_array_compatible_models import LogisticRegression_S, LinearSVM_S
from courselib.models.nn import MLP
from courselib.optimizers import GDOptimizer
import time
from IPython.display import clear_output, display
import scipy.sparse as sp
from courselib.utils.normalization import standardize
from extensions.normalization_ext import standardize_sparse_matrix

